<a href="https://colab.research.google.com/github/Hamzar2/Lab-Transformers/blob/main/Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
from farasa.segmenter import FarasaSegmenter
from farasa.diacratizer import FarasaDiacritizer
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from qalsadi.lemmatizer import Lemmatizer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizer, BertModel
import numpy as np


In [116]:
df = pd.read_csv('/content/drive/Othercomputers/My Laptop/ArabicData/scraped_data.csv' , header=None)
df.columns = ['Text']
df.head()

,Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...


## **PRE-PROCESSING DATA**


## **ADD SCORE TO DATA**

In [117]:
# nltk.download('stopwords')
df['Text_cleaned'] = df['Text'].str.replace(r'[^\u0600-\u06FF\s]+' ,'', regex=True)
stop_words = set(stopwords.words('arabic'))
df['Text_cleaned'] = df['Text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df.head()

,Text,Text_cleaned
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...


In [118]:
topic = 'نماذج اللغة الكبيرة'

# all = [topic] + [''.join(text) for i , text in df['lemm Text'].items() if text is not None]
all = [topic] + df['Text_cleaned'].tolist()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(all)
# print(tfidf_matrix[0])
# print(tfidf_matrix[1:])
cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()
for i , score in enumerate(cosine_sim):
  print(i , score)

0 0.943183346140317
1 0.0
2 0.0
3 0.1677925027550678
4 0.1521500371132732
5 0.050758226894803006
6 0.0
7 0.28415846161987623
8 0.20978231872283482
9 0.0
10 0.22467981193620282
11 0.08108417619814448
12 0.04014855229880923
13 0.1300777924361294
14 0.0
15 0.15200955033066044
16 0.061770484804173874
17 0.0976379832719534
18 0.0
19 0.0
20 0.029879171435515706
21 0.0
22 0.11915737865275773
23 0.14194659823998637
24 0.2809772702259517
25 0.09653711607156323
26 0.19457772978443577
27 0.15919478961574304
28 0.1242908461187018
29 0.19441049739300123
30 0.0
31 0.285016566435093
32 0.06113625470044178
33 0.24520342069270706
34 0.0
35 0.0
36 0.0
37 0.5880242817936283
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.310675342644293
45 0.09766576468005422
46 0.07275629130391528
47 0.05733081765724747
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.055805674118539234
55 0.0
56 0.0
57 0.0
58 0.0
59 0.031141746316102805
60 0.10277198010604235
61 0.0
62 0.0
63 0.18491868436487988
64 0.12882128485058728
65 

In [119]:
topic = 'نماذج اللغة الكبيرة'

texts = df['Text_cleaned'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text , tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

topic_embedding = get_embedding(topic , tokenizer)
text_embeddings = np.array([get_embedding(text , tokenizer) for text in texts]).squeeze(1)

relevance_scores = cosine_similarity(topic_embedding, text_embeddings).flatten()
scores = []
for i, score in enumerate(relevance_scores):
    print(f"Text {i+1} relevance score: {score}")
    scores.append(f"{score:.2f}")
df['relevance_score'] = scores
df.head()

Text 1 relevance score: 0.9755904674530029
Text 2 relevance score: 0.942656397819519
Text 3 relevance score: 0.9054958820343018
Text 4 relevance score: 0.9297724962234497
Text 5 relevance score: 0.906392514705658
Text 6 relevance score: 0.9108273983001709
Text 7 relevance score: 0.9053435921669006
Text 8 relevance score: 0.9143202304840088
Text 9 relevance score: 0.9224518537521362
Text 10 relevance score: 0.948388397693634
Text 11 relevance score: 0.9421145915985107
Text 12 relevance score: 0.8896218538284302
Text 13 relevance score: 0.9028751254081726
Text 14 relevance score: 0.9673269987106323
Text 15 relevance score: 0.917568564414978
Text 16 relevance score: 0.9329664707183838
Text 17 relevance score: 0.9052997827529907
Text 18 relevance score: 0.9474372267723083
Text 19 relevance score: 0.9091063737869263
Text 20 relevance score: 0.9106945991516113
Text 21 relevance score: 0.8983892202377319
Text 22 relevance score: 0.905248761177063
Text 23 relevance score: 0.9161394834518433
Te

,Text,Text_cleaned,relevance_score
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,0.98
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,0.94
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...,0.91
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...,0.93
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...,0.91


In [120]:
df['Text tokens'] = df['Text_cleaned'].apply(word_tokenize)
df.head()

,Text,Text_cleaned,relevance_score,Text tokens
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,0.98,"[نماذج, اللغة, الكبيرة, نماذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,0.94,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا..."
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...,0.91,"[تتكون, وحدتَي, تشفير, وفك, تشفير, قدرات, الان..."
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...,0.93,"[نماذج, اللغة, الكبيرة, القائمة, المحوِّل, قاد..."
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...,0.91,"[عكس, الشبكات, العصبونية, المتكررة, ظهرت, السا..."


In [121]:
stemmer = ISRIStemmer()
stemm  = []
for i in range(len(df['Text tokens'])):
  stemm.append([stemmer.stem(word) for word in df['Text tokens'][i]])
df['stem Text'] = stemm
df.head(2)

,Text,Text_cleaned,relevance_score,Text tokens,stem Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,0.98,"[نماذج, اللغة, الكبيرة, نماذج]","[اذج, لغة, كبر, اذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,0.94,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا...","[كبر, جدا, درب, سبق, كمي, هئل, بين, حول, سسي, ..."


In [122]:
lemmatiser = Lemmatizer()
lemm  = []
for i in range(len(df['Text tokens'])):
  lemm.append([lemmatiser.lemmatize(word) for word in df['Text tokens'][i]])
df['lemm Text'] = lemm
df.head(2)

Exception ignored in: <function ArabicDictionary.__del__ at 0x781ed4f57a30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 132078276694016 and this is thread id 132077584361024.
Exception ignored in: <function ArabicDictionary.__del__ at 0x781ed4f57a30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 132078276694016 and this is thread id 132077584361024.


,Text,Text_cleaned,relevance_score,Text tokens,stem Text,lemm Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,0.98,"[نماذج, اللغة, الكبيرة, نماذج]","[اذج, لغة, كبر, اذج]","[نماذج, لغة, كبير, نماذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,0.94,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا...","[كبر, جدا, درب, سبق, كمي, هئل, بين, حول, سسي, ...","[كبير, جد, مدرب, مسبق, كم, هائل, بيان, محول, أ..."


In [123]:
df.to_csv('/content/drive/Othercomputers/My Laptop/ArabicData/scraped_data_with_score.csv' , index=False)